In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pyodbc

server='localhost'
database='CrimeTimeDW'
username='awdemo'
password='Atlanta2022'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)

df = pd.read_sql_query("""select d.Weekday, d.Month_no, w.description as weatherdesc, w.temperature_C, f.neighborhoodKey, f.lat, f.long, f.crimeTypeKey as crime
from dimDate d JOIN weather w on d.datekey = w.DateKey
JOIN factCrime f ON f.dateKey = d.datekey 
JOIN dimCrime c ON f.crimeTypeKey = c.CrimeTypeKey
""", cnxn)
df.to_csv('datasets/MLcrimetype_all.csv')

In [101]:

# load the dataset
print("Loading Data")
crimes = pd.read_csv('datasets\\MLcrimetype_all.csv')

del crimes['Unnamed: 0']
crimes

Loading Data


,Weekday,Month_no,weatherdesc,temperature_C,neighborhoodKey,lat,long,crime
0,Wednesday,10,sky is clear,5.26,173,33.74680,-84.42540,3
1,Wednesday,10,sky is clear,5.26,218,33.72622,-84.32137,4
2,Wednesday,10,sky is clear,5.26,227,33.77073,-84.36450,3
3,Wednesday,10,sky is clear,5.26,73,33.75197,-84.43930,2
4,Wednesday,10,sky is clear,5.26,220,33.75605,-84.34813,4
...,...,...,...,...,...,...,...,...
129181,Thursday,9,sky is clear,22.35,217,33.77741,-84.33535,6
129182,Thursday,9,sky is clear,22.35,188,33.71664,-84.37816,4
129183,Thursday,9,sky is clear,22.35,28,33.85844,-84.39221,3
129184,Thursday,9,sky is clear,22.35,246,33.77800,-84.38424,3


In [102]:
# classification of categories
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(crimes['Weekday'])
crimes['Weekday'] = le.transform(crimes['Weekday'])
le.fit(crimes['weatherdesc'])
crimes['weatherdesc'] = le.transform(crimes['weatherdesc'])

crimes

,Weekday,Month_no,weatherdesc,temperature_C,neighborhoodKey,lat,long,crime
0,6,10,17,5.26,173,33.74680,-84.42540,3
1,6,10,17,5.26,218,33.72622,-84.32137,4
2,6,10,17,5.26,227,33.77073,-84.36450,3
3,6,10,17,5.26,73,33.75197,-84.43930,2
4,6,10,17,5.26,220,33.75605,-84.34813,4
...,...,...,...,...,...,...,...,...
129181,4,9,17,22.35,217,33.77741,-84.33535,6
129182,4,9,17,22.35,188,33.71664,-84.37816,4
129183,4,9,17,22.35,28,33.85844,-84.39221,3
129184,4,9,17,22.35,246,33.77800,-84.38424,3


In [103]:
# auto theft as one, others 0
condition = crimes['crime'] != 5
#crimes[condition] = crimes['crime']
crimes.loc[condition, 'crime'] = 0
crimes

,Weekday,Month_no,weatherdesc,temperature_C,neighborhoodKey,lat,long,crime
0,6,10,17,5.26,173,33.74680,-84.42540,0
1,6,10,17,5.26,218,33.72622,-84.32137,0
2,6,10,17,5.26,227,33.77073,-84.36450,0
3,6,10,17,5.26,73,33.75197,-84.43930,0
4,6,10,17,5.26,220,33.75605,-84.34813,0
...,...,...,...,...,...,...,...,...
129181,4,9,17,22.35,217,33.77741,-84.33535,0
129182,4,9,17,22.35,188,33.71664,-84.37816,0
129183,4,9,17,22.35,28,33.85844,-84.39221,0
129184,4,9,17,22.35,246,33.77800,-84.38424,0


In [104]:
crimes.loc[condition == False, 'crime'] = 1

In [105]:
# Separate features and labels
X, y = crimes[['Weekday', 'Month_no', 'weatherdesc','temperature_C','neighborhoodKey', 'lat', 'long']].values, crimes['crime'].values
print(X)
print(y)

[[  6.       10.       17.      ... 173.       33.7468  -84.4254 ]
 [  6.       10.       17.      ... 218.       33.72622 -84.32137]
 [  6.       10.       17.      ... 227.       33.77073 -84.3645 ]
 ...
 [  4.        9.       17.      ...  28.       33.85844 -84.39221]
 [  4.        9.       17.      ... 246.       33.778   -84.38424]
 [  4.        9.       17.      ... 147.       33.69033 -84.5002 ]]
[0 0 0 ... 0 0 0]


In [106]:
# Scaling data with Scaler, you need Import for scaler as below
from sklearn.preprocessing import StandardScaler
#-----------------------------------------------------------------------------

trans = StandardScaler()
X = trans.fit_transform(X)
X
# <<<<<<<<<<<<< your code here


array([[ 1.5049536 ,  0.91499101,  0.80361776, ...,  0.19770567,
        -0.22251178, -0.41477579],
       [ 1.5049536 ,  0.91499101,  0.80361776, ...,  0.77120527,
        -0.6813619 ,  1.82958656],
       [ 1.5049536 ,  0.91499101,  0.80361776, ...,  0.88590519,
         0.31102968,  0.89909201],
       ...,
       [ 0.50682524,  0.63166943,  0.80361776, ..., -1.65023749,
         2.2666052 ,  0.30127137],
       [ 0.50682524,  0.63166943,  0.80361776, ...,  1.12804947,
         0.47312105,  0.47321762],
       [ 0.50682524,  0.63166943,  0.80361776, ..., -0.13364965,
        -1.48156263, -2.02852474]])

In [107]:
# Split data into training set and test set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)


In [108]:
# Set regularization hyperparameter
reg = 0.1
# Use this model for prediction: LogisticRegression(C=1/reg, solver="liblinear")

model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)




In [109]:
# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)


Accuracy: 0.862550314789968


In [110]:
# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
#auc = roc_auc_score(y_test,y_scores, multi_class='ovr')

print('AUC: ' + str(auc))


AUC: 0.5885481448419496


In [111]:
# load dataset with 5 new rows
print("Loading Data")
predict = pd.read_csv('datasets\\MLcrimetype_topredict.csv')

del predict['Unnamed: 0']
del predict['crime']

# print rows to predict

predict_fitted = predict.copy()
predict = predict[-5:]
predict

Loading Data


,Weekday,Month_no,weatherdesc,temperature_C,neighborhoodKey,lat,long
129187,Monday,7,scattered clouds,21.35,73,33.71893,-84.33940
129188,Thursday,10,sky is clear,22.35,67,33.78558,-84.45560
129189,Monday,4,sky is clear,12.35,200,33.73686,-84.37883
129190,Thursday,8,sky is clear,22.35,197,32.70432,-84.57693
129191,Tuesday,10,mist,17.35,200,33.72687,-84.37883


In [112]:
# classify categories and fit the data
le.fit(predict_fitted['Weekday'])
predict_fitted['Weekday'] = le.transform(predict_fitted['Weekday'])
le.fit(predict_fitted['weatherdesc'])
predict_fitted['weatherdesc'] = le.transform(predict_fitted['weatherdesc'])

predict_fitted = trans.fit_transform(predict_fitted)
predict_fitted = predict_fitted[-5:]
predict_fitted

array([[ -0.99038021,   0.06501971,   0.62082308,   0.9707318 ,
         -1.07672799,  -0.84040923,   1.43814909],
       [  0.50681378,   0.91499541,   0.80360736,   1.09105818,
         -1.15319494,   0.63949417,  -1.064741  ],
       [ -0.99038021,  -0.78495599,   0.80360736,  -0.11220566,
          0.54182238,  -0.44228968,   0.58884654],
       [  0.50681378,   0.34834494,   0.80360736,   1.09105818,
          0.50358891, -23.36891316,  -3.67812873],
       [  1.00587844,   0.91499541,  -0.11031403,   0.48942626,
          0.54182238,  -0.66410866,   0.58884654]])

In [124]:
#### predict case

y_pred = model.predict(predict_fitted)
print(y_pred)
print('Car stolen:')
for x in y_pred:
    if x == 1:
        print(predict.iloc[np.where(y_pred==x)])

[0 0 0 1 0]
Car stolen:
         Weekday  Month_no   weatherdesc  temperature_C  neighborhoodKey  \
129190  Thursday         8  sky is clear          22.35              197   

             lat      long  
129190  32.70432 -84.57693  
